In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
#Import Visualization libraries 
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# project path
path = r"C:\Users\richa\Documents\01-2026 Instacart Basket Analysis"

# Load customer data
customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

# Load merged orders_products data
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merged_flags.pkl')) 


In [3]:
##customer dataset
customers.columns

Index(['user_id', 'First Name', 'Surnam', 'Gender', 'STATE', 'Age',
       'date_joined', 'n_dependants', 'fam_status', 'income'],
      dtype='object')

In [4]:
 customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
customers.dtypes

user_id          int64
First Name      object
Surnam          object
Gender          object
STATE           object
Age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [6]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


In [7]:
#4 wraggling and cleaning customer data
# Rename columns for consistency
customers.rename(columns={
    'First Name': 'first_name',
    'Surnam': 'surname',
    'Gender': 'gender',
    'STATE': 'state',
    'Age': 'age',
    'date_joined': 'date_joined',
    'n_dependants': 'n_dependents',
    'fam_status': 'fam_status',
    'income': 'income'
}, inplace=True)

In [8]:
#converting dae_joined from object to integer
customers['date_joined'] = pd.to_datetime(customers['date_joined'], errors='coerce')

In [9]:
#duplicate check
customers.duplicated().sum()

np.int64(0)

In [10]:
#missing value
customers.isnull().sum()

user_id             0
first_name      11259
surname             0
gender              0
state               0
age                 0
date_joined         0
n_dependents        0
fam_status          0
income              0
dtype: int64

In [11]:
# first_name has 11259 missing values. replace it with unkown

In [12]:
# Fill missing first names with "Unknown"
customers['first_name'] = customers['first_name'].fillna('Unknown')

In [13]:
#re-check missing values
customers.isnull().sum()

user_id         0
first_name      0
surname         0
gender          0
state           0
age             0
date_joined     0
n_dependents    0
fam_status      0
income          0
dtype: int64

In [14]:
#merging ords_prods with customer data with inner join
ords_prods_merged_customers = ords_prods_merge.merge(
                                customers, 
                                on='user_id', 
                                how='inner')

In [15]:
#check
ords_prods_merged_customers.shape

(32404859, 34)

In [16]:
ords_prods_merged_customers.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_dow,order_hour_of_day,...,order_frequency_flag,first_name,surname,gender,state,age,date_joined,n_dependents,fam_status,income
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,Frequent customer,Charles,Cox,Male,Minnesota,81,2019-08-01,1,married,49620
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,Frequent customer,Charles,Cox,Male,Minnesota,81,2019-08-01,1,married,49620
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,Frequent customer,Deborah,Glass,Female,Vermont,66,2018-06-16,2,married,158302
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,Frequent customer,Heather,Myers,Female,Wisconsin,40,2020-02-09,3,married,31308
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,Frequent customer,Heather,Myers,Female,Wisconsin,40,2020-02-09,3,married,31308


In [26]:
ords_prods_merged_customers.isnull().sum()

product_id                       0
product_name                     0
aisle_id                         0
department_id                    0
prices                           0
order_id                         0
user_id                          0
order_number                     0
order_dow                        0
order_hour_of_day                0
days_since_prior_order     2076096
add_to_cart_order                0
reordered                        0
_merge                           0
merge_indicator                  0
price_range_loc           21208222
Busiest day                      0
busiest_days_new                 0
busiest_period_of_day            0
first_name                       0
surname                          0
gender                           0
state                            0
age                              0
date_joined                      0
n_dependents                     0
fam_status                       0
income                           0
dtype: int64

In [19]:
def reduce_memory_usage(df):
    import numpy as np
    import pandas as pd

    start_mem = df.memory_usage().sum() / 1024**2
    print(f"Initial memory usage: {start_mem:.2f} MB")

    for col in df.columns:
        col_type = df[col].dtype

        if pd.api.types.is_integer_dtype(col_type):
            df[col] = pd.to_numeric(df[col], downcast='integer')
        elif pd.api.types.is_float_dtype(col_type):
            df[col] = pd.to_numeric(df[col], downcast='float')
        elif col_type == object:
            num_unique_values = df[col].nunique()
            num_total_values = len(df[col])
            if num_unique_values / num_total_values < 0.5:
                df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print(f"Reduced memory usage: {end_mem:.2f} MB")
    print(f"Memory decreased by {(start_mem - end_mem) / start_mem * 100:.1f}%")
    return df

# Drop unnecessary columns before optimization
cols_to_drop = ['_merge', 'merge_indicator'] 
ords_prods_merged_customers = ords_prods_merged_customers.drop(columns=cols_to_drop)

# Downcast and convert categories
ords_prods_merged_customers = reduce_memory_usage(ords_prods_merged_customers)


Initial memory usage: 3616.11 MB
Reduced memory usage: 2040.07 MB
Memory decreased by 43.6%


In [20]:
print(ords_prods_merged_customers.memory_usage(deep=True) / 1024**2)

Index                        0.000126
product_id                  61.807364
product_name               127.417903
aisle_id                    30.903682
department_id               30.903682
prices                     123.614727
order_id                   123.614727
user_id                    123.614727
order_number                30.903682
order_dow                   30.903682
order_hour_of_day           30.903682
days_since_prior_order     123.614727
add_to_cart_order           30.903682
reordered                   30.903682
price_range_loc             30.903872
Busiest day                 30.903855
busiest_days_new            30.903856
busiest_period_of_day       30.903858
max_order                   30.903682
loyalty_flag                30.903862
mean_price                 123.614727
spending_flag               30.903797
median_days_since_prior    123.614727
order_frequency_flag        30.903873
first_name                  61.826131
surname                     61.891440
gender      

In [22]:
# Export merged dataframe as pickle
ords_prods_merged_customers.to_pickle(
    os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merged_customers.pkl'))
